In [1]:
from elec_bol20 import *
import elec_bol20.util as ebu


In [2]:
path = 'datos_1_intermedios/2020/clustermodel/clustermodel_mesas_percentage_actual.csv'
path = os.path.join(ebu.DIR,path)
df = pd.read_csv(path,dtype={'ID_MESA':str})

In [3]:
df['ID_MESA'].isna().sum()

5

In [4]:
df.set_index('ID_MESA')

,cluster,HAB,BL_perc_mean,NU_perc_mean,MAS_perc_mean,CC_perc_mean,CREEMOS_perc_mean,FPV_perc_mean,PAN_BOL_perc_mean,vv_mean
ID_MESA,,,,,,,,,,
100320127940021102,1.0,220.0,0.903648,4.377883,33.356780,43.406303,21.293442,1.523109,0.406871,84.642491
100320127940021103,1.0,220.0,0.903648,4.377883,33.356780,43.406303,21.293442,1.523109,0.406871,84.642491
100320127940021104,1.0,220.0,0.903648,4.377883,33.356780,43.406303,21.293442,1.523109,0.406871,84.642491
100320127940021105,1.0,220.0,0.903648,4.377883,33.356780,43.406303,21.293442,1.523109,0.406871,84.642491
100320127940021107,1.0,220.0,0.903648,4.377883,33.356780,43.406303,21.293442,1.523109,0.406871,84.642491
...,...,...,...,...,...,...,...,...,...,...
NaN,NaN,NaN,1.098583,4.171408,36.074891,43.732110,17.901897,1.795564,0.489233,84.063308
NaN,NaN,NaN,1.098583,4.171408,36.074891,43.732110,17.901897,1.795564,0.489233,84.063308
NaN,NaN,NaN,1.098583,4.171408,36.074891,43.732110,17.901897,1.795564,0.489233,84.063308


In [5]:
df['ID_RECI'] = df['ID_MESA'].str[:-2]
df = df[~df['ID_RECI'].isna()].set_index('ID_RECI')
df.index = df.index.astype(np.int64)

In [6]:
cart = os.path.join(ebu.DATA_PATH1_2020,'z030_carto_xy.csv')
cart = pd.read_csv(cart).set_index('ID_RECI')
cart.index = cart.index.astype(np.int64)

cart = os.path.join(ebu.DATA_PATH1_2020,'z030_carto_xy.csv')
cart = pd.read_csv(cart).set_index('ID_RECI')
cart.index = cart.index.astype(np.int64)


_mean = ['X', 'Y']
_sum = ['BL_perc_mean', 'NU_perc_mean',
       'MAS_perc_mean', 'CC_perc_mean', 'CREEMOS_perc_mean', 'FPV_perc_mean',
       'PAN_BOL_perc_mean', 'vv_mean']

df[_sum] =  df[_sum].multiply(df['HAB'],axis='index')

In [7]:
dj = df.join(cart)
l = len(dj)
gr = dj.groupby('ID_RECI') 

In [8]:
nd = gr[_mean].mean()
nd[_sum] = gr[_sum].sum()
nd['HAB'] = gr['HAB'].sum()
nd[_sum] = nd[_sum].multiply(1/nd['HAB'],axis='index')



In [9]:
nd['MAS_perc_mean']

ID_RECI
1003201120000001    61.383679
1003201120000002    49.272735
1003201120400052    90.932570
1003201120500053    86.268170
1003201120600054    84.550823
                      ...    
1119472381525140    33.356780
1121266380926124    33.433940
1121377382026120    12.951380
1123349379225117    43.595777
1123750379325118          NaN
Name: MAS_perc_mean, Length: 5538, dtype: float64

In [10]:
va = 'CREEMOS_perc_mean'
def plo(va):
    C_DIC = {
        'CC_perc_mean':ebu.P_GRAD_CC,
        'FPV_perc_mean':ebu.P_GRAD_FPV,
        'MAS_perc_mean':ebu.P_GRAD_MAS,
        'CREEMOS_perc_mean':ebu.P_GRAD_CREEMOS,
        'PAN_BOL_perc_mean':ebu.P_GRAD_PANBOL,
    }



    MAX = nd[va].quantile(.99)

    nd['s'] = nd['HAB']/1000
    sr = bokeh.models.ColumnDataSource(nd)
    from bokeh.transform import linear_cmap
    from bokeh.transform import log_cmap

    cm = linear_cmap(va, palette=C_DIC[va], low=0, high=MAX)

    cb = bokeh.models.ColorBar(
        color_mapper=cm['transform'])

    f = bokeh.plotting.figure()
    f.scatter(x='X',y='Y',source = sr,color=cm,size = 's')
    f.add_layout(cb)
    f.title.text=va
    TOOL_TIP = [
        ('Inscritos', '@HAB'),
        ('PAIS, Municipalidad', '@PAIS, @MUN'),
        ('Recinto', '@REC'),
        (f'{va} [%]', f'@{va}{{0.0}}'),
    #     ('CC [%]','@cc{0.0}'),
    #     ('Diferencia [%]', '@ad_mas_cc{0.0} (@mas_o_cc)'),
        ('------','------')
        # ('DEN %', '@DEN')
        # ('PAIS', '@PAIS'),
    ]

    ho = bokeh.models.HoverTool(
    tooltips=TOOL_TIP
    )

    f.add_tools(ho)

    bokeh.plotting.output_notebook()
    bokeh.plotting.show(f)

ll=['MAS_perc_mean', 'CC_perc_mean', 'CREEMOS_perc_mean', 'FPV_perc_mean',
       'PAN_BOL_perc_mean']
    
for va in ll: plo(va)

Loading BokehJS ...

Loading BokehJS ...